In [ ]:
from physcraper import opentree_helpers
from opentree import OT
import dendropy
import json
import sys


In [ ]:
# Input alignemnt from treebase stored at https://github.com/McTavishLab/physcraper_example/blob/master/treebase_alns/pg_55tree5864_ndhf.aln
# Data files created by running:
# physcraper_run.py -s pg_55 -t tree5864 -a treebase_alns/pg_55tree5864_ndhf.aln -as "nexus" -db ~/ncbi/localblastdb/ -o pg_55


In [ ]:
updated_tre_path = "pg_55/outputs_pg_55tree5864_ndhf/physcraper_pg_55tree5864_ndhf.tre"
otu_json_path = "pg_55/run_pg_55tree5864_ndhf/otu_info_pg_55tree5864_ndhf.json"

In [ ]:
updated_tre = dendropy.Tree.get(path=updated_tre_path, schema = "newick")
otu_dict = json.load(open(otu_json_path, "r"))

In [ ]:
#Choose outgroups and root
outgroup = ['otu376420','otu376439','otu376452']
mrca = updated_tre.mrca(taxon_labels=outgroup)
updated_tre.reroot_at_node(mrca, update_bipartitions=True)


In [ ]:
tree_conf = opentree_helpers.conflict_tree(updated_tre, otu_dict)
print(tree_conf)

In [ ]:
resp_updated = OT.conflict_str(tree_conf.as_string(schema="newick"), 'ott')

In [ ]:
conflict = resp_updated.response_dict
conflict

In [ ]:


updated_conf_taxa = set()
for node in conflict:
    if conflict[node]['status'] == 'conflicts_with':
        witness = conflict[node]['witness_name']
        updated_conf_taxa.add(witness)

sys.stdout.write("Updated tree conflicts with {} taxa in the OpenTree taxonomy:\n".format(len(updated_conf_taxa)))
for tax in updated_conf_taxa:
    sys.stdout.write("{}\n".format(tax))


In [ ]:
## Relabel tree with conflict information
for node in tree_conf:
    if node.taxon:
        node_id = node.taxon.label.split('_')[1]
        conf_node = conflict.get(node_id, False)
        if conf_node:
            new_label = "{} {}".format(conf_node['status'], conf_node['witness_name'])
            node.taxon.label = new_label
    else:
        node_id = node.label.split('_')[1]
        conf_node = conflict.get(node_id, False)
        if conf_node:
            new_label = "{} {}".format(conf_node['status'], conf_node['witness_name'])
            node.label = new_label


In [ ]:
tree_conf.as_string(schema="newick")

In [ ]:
tree_conf.as_ascii_plot()